In [1]:
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
from itertools import groupby
from operator import itemgetter

import numpy as np
from scipy import stats
from datascience import *
from scipy.signal import savgol_filter
from scipy import stats 
from scipy.integrate import simps
import subprocess
from io import BytesIO
import pandas as pd

read_data = Table.read_table("Thagomizer_Runs/icos-cd4.csv")
chrom = "chr1"
start = read_data.column(0).item(0)
stop = read_data.column(0).item(read_data.num_rows - 1)
columns = read_data.num_columns

def MA_normalization(read_data):
    M = np.log(read_data.column(1) + .999999) - np.log(read_data.column(2) + .999999)
    A = (np.log(read_data.column(1) + .999999) + np.log(read_data.column(2) + .999999))
    slope, intercept, r_value, p_value, std_err =  stats.linregress(A,M)
    adjusted_M = M - ((slope*A)+intercept)
    return Table().with_columns("Adjusted M", adjusted_M, "A", A)

def ranges(nums):
    nums = sorted(set(nums))
    gaps = [[s, e] for s, e in zip(nums, nums[1:]) if s+1 < e]
    edges = iter(nums[:1] + sum(gaps, []) + nums[-1:])
    summ = list(zip(edges, edges))
    start = [summ[x][0] for x in np.arange(len(summ))]
    stop = [summ[x][1] for x in np.arange(len(summ))]
    
    return Table().with_columns("start", start, "stop", stop, "range", [i-j for i,j in zip(stop,start)])

def tbl_generator(upper,data_comp):
        upper = upper
        lower = -upper
        u = np.mean(data_comp.column(3))
        o = np.std(data_comp.column(3))
        summary_tbl = data_comp.with_column("Greater Binding in Condition 1", ((data_comp.column(3)-u)/o)>upper)
        summary_tbl = summary_tbl.with_column("Greater Binding in Condition 2", ((data_comp.column(3)-u)/o)<lower)

        cond_1 = ranges(summary_tbl.where("Greater Binding in Condition 1", are.equal_to(True)).column(0))
        cond_2 = ranges(summary_tbl.where("Greater Binding in Condition 2", are.equal_to(True)).column(0))
        return cond_1.where("range", are.above(3)), cond_2.where("range", are.above(3)), summary_tbl

In [2]:
def regions(read_data,chrom,start,stop):
    data_comp = read_data.with_column("Adjusted_Fitted_M", savgol_filter(MA_normalization(read_data).column("Adjusted M"), 21,1))
    data_comp_mv = data_comp
    data_comp_old = data_comp
    Adjusted_Fitted_M = data_comp.column(3)
    data_comp = data_comp.select(0,1,2).with_column("Adjusted_Fitted_Normalized_M", (data_comp.column(3)- np.mean(data_comp.column(3))/(np.std(data_comp.column(3)))))
    z_score = 1.75
    cond1, cond2, summ_tbl = tbl_generator(z_score,data_comp)
    true_difference = MA_normalization(read_data).column("Adjusted M")
    fin_tbl = Table().with_columns("pos", summ_tbl.column(0), "diff", true_difference)
    
    cond1_differences = make_array()
    for i in np.arange(cond1.num_rows):
        x = cond1.column(0).item(i)
        y = cond1.column(1).item(i)
        differences = fin_tbl.where("pos", are.between(x,y + 1)).column("diff")
        under_curve = simps(np.abs(differences))
        cond1_differences = np.append(cond1_differences, under_curve)
    cond1 = cond1.with_column("increased binding (unitless)", cond1_differences)
    
    cond2_differences = make_array()
    for i in np.arange(cond2.num_rows):
        x = cond2.column(0).item(i)
        y = cond2.column(1).item(i)
        differences = fin_tbl.where("pos", are.between(x,y + 1)).column("diff")
        under_curve = simps(np.abs(differences))
        cond2_differences = np.append(cond2_differences, under_curve)
    cond2 = cond2.with_column("increased binding (unitless)", cond2_differences)
    
    return cond1.column(0), cond1.column(1), cond1.column(2), cond1.column(3), cond2.column(0), cond2.column(1), cond2.column(2), cond2.column(3)


In [3]:
def get_depth_data(track_files,track_names,chrom,start,stop,strand,track_type):
    def view_region(track_file,strand,region):
        return subprocess.Popen(("samtools", "view",
                                 strand_to_flag[use_strand],
                                 "-b",track_file,
                                 region),stderr=subprocess.PIPE,stdout=subprocess.PIPE)
    mydepths = pd.DataFrame([0]*(stop-start+1),index=range(start,stop+1),columns=["depth"])
    depth_list = pd.DataFrame(0,index=range(start,stop),columns=track_names)
    strandinvert = {"+":"-","-":"+"}
    strand_to_flag = {"+":"-F 0x10",
        "-":"-f 0x10"}
    for n,track_file in enumerate(track_files):
        use_strand=strand
        region = chrom + ":" + str(start) + "-" + str(stop)
        if track_type[n] == "as":
            use_strand = strandinvert[strand]
        # Get sequences from a given region (in binary bam format still)
        ps =view_region(track_file,strand_to_flag[use_strand],region)
        sout,err = ps.communicate() # get stdout, stderr
        ## CHECK TO MAKE SURE THE REFERENCE GENOME CHROMOSOME IS FINE.
        if len(err)>0: # is there anytihn in stder?
            if b"specifies an unknown reference name" in err:
                # SWITCH REFERENCE
                temp_chrom = chrom.replace("chr","")
                region = temp_chrom + ":" + str(start) + "-" + str(stop)
                ps =view_region(track_file,strand_to_flag[use_strand],region)
                sout,err = ps.communicate()
        if len(err)>0:
            raise NameError("Unknown samtools error. Ran: samtools view %s -b %s %s | samtools depth - " % (strand_to_flag[use_strand],track_file,region))
        # Run samtools depth on the sequences retrieved
        ps2 = subprocess.Popen(("samtools", "depth","-"),stdin=subprocess.PIPE,stdout=subprocess.PIPE)
        output,err = ps2.communicate(input=sout)
        sample_depths = pd.read_table(BytesIO(output),names=["chrom","depth"],index_col=1)
        if len(sample_depths.index)>0:
            mydepths.depth = sample_depths.depth
            depth_list[track_names[n]] = sample_depths.depth
            depth_list = depth_list.fillna(value=0)  
    return depth_list

In [ ]:
#cond1,cond2 = regions(read_data,chrom,start,stop)
unique_utrs = Table.read_table("refseq_3utr.csv").drop("Num")
f1 = "bams/Loeb_ko_sorted.bam"
f2 = "bams/Loeb_wt_sorted.bam"

cond1_starts = make_array()
cond1_stops = make_array()
cond1_ranges = make_array()
cond1_binds = make_array()
cond1_strands = make_array()
cond1_chroms = make_array()
cond1_geneIDs = make_array()

cond2_starts = make_array()
cond2_stops = make_array()
cond2_ranges = make_array()
cond2_binds = make_array()
cond2_strands = make_array()
cond2_chroms = make_array()
cond2_geneIDs = make_array()


import time

from tqdm import tqdm
for i in tqdm(np.arange(unique_utrs.num_rows)):
    #unique_utrs = unique_utrs.where("GeneID", are.equal_to("Actb"))
    chrom = unique_utrs.column(0).item(i)
    start = unique_utrs.column(1).item(i)
    stop = unique_utrs.column(2).item(i)
    strand = unique_utrs.column(4).item(i)
    geneid = unique_utrs.column(3).item(i)

    region = chrom + ":" + str(start) + "-" + str(stop)
    #strand_to_flag = {"+":"-F 0x10","-":"-f 0x10"}
    #ps = subprocess.Popen(("samtools", "view",strand_to_flag[strand],"-b",f1,region),stderr=subprocess.PIPE,stdout=subprocess.PIPE)
    #sout,err = ps.communicate()
    #sample_depths = pd.read_table(BytesIO(output),names=["chrom","depth"],index_col=1)
    #ps2 = subprocess.Popen(("samtools", "depth","-"),stdin=subprocess.PIPE,stdout=subprocess.PIPE)
    #output,err = ps2.communicate(input=sout)
    #sample_depths = pd.read_table(BytesIO(output),names=["chrom","depth"],index_col=1)

    depths = get_depth_data(make_array(f1, f2),make_array(f1, f2),chrom,start,stop,strand,["s", "s"])
    f1_depths = depths[f1].tolist()
    f2_depths = depths[f2].tolist()

    reading_data = Table().with_columns("Unnamed: 0",depths.index.tolist(), f1,f1_depths, f2, f2_depths)
    if sum(reading_data.column(1)) > 0 and sum(reading_data.column(2)) > 0 and reading_data.num_rows >21:
        cond1_start, cond1_stop, cond1_range, cond1_bind, cond2_start, cond2_stop, cond2_range, cond2_bind = regions(reading_data,chrom,start,stop)
        
        #cond1_start = [int(i) for i in cond1_start]
        #cond1_stop = [int(i) for i in cond1_stop]
        
        cond2_start = [int(i) for i in cond2_start]
        cond2_stop = [int(i) for i in cond2_stop]
        
        cond1_geneID = [geneid for i in np.arange(len(cond1_start))]
        cond2_geneID = [geneid for i in np.arange(len(cond2_start))]

        cond1_strand = [strand for i in np.arange(len(cond1_start))]
        cond2_strand = [strand for i in np.arange(len(cond2_start))]

        cond1_chrom = [chrom for i in np.arange(len(cond1_start))]
        cond2_chrom = [chrom for i in np.arange(len(cond2_start))]

        cond1_starts = np.append(cond1_starts, cond1_start)
        cond1_stops = np.append(cond1_stops, cond1_stop)
        cond1_binds = np.append(cond1_binds, cond1_bind)
        cond1_strands = np.append(cond1_strands, cond1_strand)
        cond1_chroms = np.append(cond1_chroms, cond1_chrom)
        cond1_geneIDs = np.append(cond1_geneIDs, cond1_geneID)

        cond2_starts = np.append(cond2_starts, cond2_start)
        cond2_stops = np.append(cond2_stops, cond2_stop)
        cond2_binds = np.append(cond2_binds, cond2_bind)
        cond2_strands = np.append(cond2_strands, cond2_strand)
        cond2_chroms = np.append(cond2_chroms, cond2_chrom)
        cond2_geneIDs = np.append(cond2_geneIDs, cond2_geneID)
        
        
cond1_summary = Table().with_columns("chrom",cond1_chroms, "chromStart",cond1_starts,  "chromEnd",cond1_stops,  "name",cond1_geneIDs, "strand",cond1_strands, "differential binding (unitless)", cond1_binds)
cond2_summary = Table().with_columns("chrom",cond2_chroms, "chromStart",cond2_starts,  "chromEnd",cond2_stops,  "name",cond2_geneIDs, "strand",cond2_strands, "differential binding (unitless)", cond2_binds)
cond2_summary

 48%|████▊     | 10003/20765 [33:03<21:57,  8.17it/s]  /anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:52: RuntimeWarning: invalid value encountered in greater
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in less
 99%|█████████▉| 20521/20765 [1:14:16<00:46,  5.23it/s]  

In [ ]:
dclip = Table.read_table("dCLIP-1.7-run-1-int/dCLIP_output.txt")
#sites = dclip.where("chrom", are.equal_to(chrom)).where("position", are.between(start, stop + 1)).select("chrom", "position", "state")

In [ ]:
d_cond_1 = dclip.where("state", are.equal_to(2))#.where("position",are.between(limit.item(0), limit.item(1)))
d_cond_2 = dclip.where("state", are.equal_to(0))#.where("position",are.between(limit.item(0), limit.item(1)))

utrs = Table.read_table("3'utr.csv").drop("Num")
utrs = utrs.with_column("range", utrs.column("Stop") - (utrs.column("Start")-1))#.group(make_array(0,1,2,3,4), max)
unique_names = utrs.group("GeneID").column(0)

chroms = make_array()
starts = make_array()
stops = make_array()
geneids = make_array()
strands = make_array()
ranges = make_array()


for i in np.arange(len(unique_names)):
    utr = utrs.where("GeneID", are.equal_to(unique_names.item(i))).sort("range", descending=True).take(0)
    
    chroms = np.append(chroms,utr.column("Chrom").item(0))
    starts = np.append(starts,utr.column("Start").item(0))
    stops = np.append(stops,utr.column("Stop").item(0))
    geneids = np.append(geneids,utr.column("GeneID").item(0))
    strands = np.append(strands,utr.column("Strand").item(0))
    ranges = np.append(ranges,utr.column("range").item(0))
unique_utrs = Table().with_columns("Chrom", chroms, "Start", starts, "Stop", stops, "GeneID", geneids, "Strand",strands, "Range", ranges)
unique_utrs = Table().with_columns("Chrom", unique_utrs.column("Chrom"), "Start", [int(i) for i in unique_utrs.column("Start")], "Stop", [int(i) for i in unique_utrs.column("Stop")], "GeneID",unique_utrs.column("GeneID"), "Num",[int(i) for i in np.zeros(unique_utrs.num_rows)], "Strand", unique_utrs.column("Strand"))